### CLARENCE
#### Notebook primarily for testing components 

### 18 Jun 
Pipeline not running as expected

In [21]:
import json
import pathlib

from google_cloud_pipeline_components.experimental.custom_job.utils import (
    create_custom_training_job_op_from_component,
)

from kfp.v2 import compiler, dsl
from pipelines.kfp.helpers import generate_query

from pipelines.kfp_components.ingest import bq_extract_data, bq_query_to_table
from pipelines.kfp_components.training import train_xgboost_model
# from pipelines.kfp_components.dependencies import PROJECT_ID, ROOT_DIR, sql_query

print(ROOT_DIR)

@dsl.pipeline(name="xgboost-train-pipeline")
def old_xgboost_pipeline(
    # project_id: str,
    # project_location: str,
    # pipeline_files_gcs_path: str,
    # ingestion_project_id: str,
    # tfdv_schema_filename: str,
    # tfdv_train_stats_path: str,
    # model_name: str,
    # model_label: str,
    # dataset_id: str,
    # dataset_location: str,
    # ingestion_dataset_id: str,
    # timestamp: str,
):
    """
    Query a view from BQ
    Extract the view to GCS

    """

    sql_query = generate_query(
        input_file=ROOT_DIR
        / "pipelines"
        / "kfp"
        / "ingest"
        / "queries"
        / "query_bq.sql"
    )

    ingest = bq_query_to_table(
        query=sql_query,
        bq_client_project_id=None,
        destination_project_id=PROJECT_ID,
        dataset_id="dwh_pacific_torus",
        table_id="credit_card_default",
        dataset_location="US",
        query_job_config=None,
    )
    # .set_display_name("Ingest data")

    # ingest_to_gcs = (
    #     bq_extract_data(
    #         source_project_id="pacific-torus-347809",
    #         source_dataset_id="dwh_pacific_torus",
    #         source_table_id="credit_card_default",
    #         destination_project_id="pacific-torus-347809",
    #         destination_bucket="mle-dwh-torus",
    #         destination_file="raw/credit_cards.csv",
    #         dataset_location="US",
    #     )
    #     # .after(ingest)
    #     # .set_display_name("Export to GCS")
    # )


def compile():

    compiler.Compiler().compile(
        pipeline_func=xgboost_pipeline,
        pipeline_name="xgboost-train-pipeline",
        package_path="training.yaml",
        type_check=True,
    )


if __name__ == "__main__":

    custom_train_job = create_custom_training_job_op_from_component(
        component_spec=train_xgboost_model,
        replica_count=1,
        machine_type="n1-standard-4",
    )

    compile()

ModuleNotFoundError: No module named 'pipelines.kfp.dependencies'

### 22 Jun Test ingest component step

In [2]:
import os
import kfp
from kfp.v2 import compiler, dsl

In [3]:
client = kfp.Client()

Failed to load kube config.


MaxRetryError: HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: /apis/v1beta1/healthz (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb8ac2c78e0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [38]:
ingest_op = kfp.components.load_component_from_file("../ingest_component.yaml")
tfdv_op = kfp.components.load_component_from_file("../tfdv_component.yaml")

In [24]:
## ingest_op.component_spec

In [39]:
# Define the pipeline
@dsl.pipeline(
   name='testing pipeline',
   description='NIL'
)
def xgboost_test_pipeline(
):
    ingest = ingest_op(
        source_project_id="pacific-torus-347809",
        source_dataset_id="dwh_pacific_torus",
        source_table_id="credit_card_defaults",
        destination_project_id="pacific-torus-347809",
        destination_bucket="mle-dwh-torus",
        destination_file="raw/credit_cards.csv",
        dataset_location="US",
        extract_job_config="None",
    ) # .apply(gcp.use_gcp_secret('user-gcp-sa'))

    stats = tfdv_op(  # TFDV stats for the test data
        input_data="gs://mle-dwh-torus/raw/credit_cards.csv",
        output_path="gs://mle-dwh-torus/tfdv_expers/eval/evaltest.pb",
        job_name='test-1',
        use_dataflow="False",
        project_id="pacific-torus-347809", 
        region="US",
        gcs_temp_location='gs://mle-dwh-torus/tfdv_expers/tmp',
        gcs_staging_location='gs://mle-dwh-torus/tfdv_expers',
        whl_location="None", 
        requirements_file="requirements.txt"
    ).after(ingest)

In [41]:
compiler.Compiler().compile(
    pipeline_func=xgboost_test_pipeline,
    pipeline_name="xgboost-train-pipeline",
    package_path="test.json",
    type_check=True,
)

### 23 Jun Simple pipeline trial

In [271]:
from typing import NamedTuple
from kfp.v2.dsl import Dataset, Output, component, OutputPath, Artifact, Input
import google.cloud.aiplatform as aip
from kfp.v2 import compiler, dsl

aip.init(project="pacific-torus-347809", staging_bucket="gs://mle-dwh-torus")

In [272]:
def bq_extract_data(
    source_project_id: str,
    source_table_url: str,
    destination_project_id: str,
    destination_bucket: str, 
    destination_file: str,
    dataset_location: str,
    extract_job_config: dict = None,
) -> NamedTuple('outputs', [('dataset_gcs_uri', str), 
    ('dataset_gcs_directory', str)]
):

    import logging
    import os 
    from google.cloud.exceptions import GoogleCloudError
    from google.cloud import bigquery, storage


    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "pacific-torus.json"

    # logging.config.fileConfig(LOGGING_CONF)
    logger = logging.getLogger("root")

    storage_client = storage.Client(project=destination_project_id)

    if not storage.Bucket(storage_client, destination_bucket).exists():
        bucket = storage_client.bucket(destination_bucket)
        bucket.storage_class = "STANDARD" 
        storage_client.create_bucket(
            bucket_or_name=bucket, location="ASIA-SOUTHEAST1", project=destination_project_id
        )
        logger.info(f"Bucket created {destination_bucket}")

    full_table_url = f"{source_project_id}.{source_table_url}"
    table = bigquery.table.Table(table_ref=full_table_url)

    if extract_job_config is None:
        extract_job_config = {}
    if destination_file.endswith(".json"):
        extract_job_config = {"destination_format": "NEWLINE_DELIMITED_JSON"}
    job_config = bigquery.ExtractJobConfig(**extract_job_config)


    dataset_gcs_uri = f"gs://{destination_bucket}/{destination_file}"
 
    bq_client = bigquery.Client(project=destination_project_id)

    logger.info(f"Extract {source_table_url} to {dataset_gcs_uri}")
    extract_job = bq_client.extract_table(
        source=table,
        destination_uris=dataset_gcs_uri,
        job_config=job_config,
        location=dataset_location,
    )

    dataset_gcs_directory = os.path.dirname(dataset_gcs_uri)

    try:
        extract_job.result()
        logger.info(f"Table extracted: {dataset_gcs_uri}")
    except GoogleCloudError as e:
        logger.error(e)
        logger.error(extract_job.error_result)
        logger.error(extract_job.errors)
        raise e

    return (dataset_gcs_uri, dataset_gcs_directory)

In [410]:
def basic_preprocessing(
    input_file: str,
    output_bucket: str,
    output_file: str
) -> Artifact: 

    import pandas as pd
    from functools import reduce

    df = pd.read_csv(input_file)

    df.columns = [col.lower().strip() for col in df.columns] 

    df.dropna(inplace=True)
    
    df.rename({'default_payment_next_month': "default"}, inplace=True)
    df.loc[df['education'] == '0', 'education'] = 'Unknown'
    df.loc[df['marriage'] == '0', 'marriage'] = 'Other'
    sex = pd.get_dummies(df.sex, prefix='gender')
    education = pd.get_dummies(df.education, prefix='ed')
    marriage = pd.get_dummies(df.marriage, prefix='mstatus')
    frames = [df, sex, education, marriage]
    final = reduce(lambda l, r: pd.concat([l, r], axis=1), frames)
    final.drop(['default_payment_next_month', 'sex', 'education', 'marriage'], axis=1, inplace=True)

    output_path = f"gs://mle-dwh-torus/{output_bucket}/{output_file}" 
    final.to_csv(output_path, index=False)

    return final


def train_test_split(
    input_file: InputPath("CSV"),
    output_bucket: str,
) -> NamedTuple("outputs", [
    ("train_data", Artifact),
    ("test_data", Artifact)
]):

    import pandas as pd
    from sklearn.model_selection import train_test_split

    df = pd.read_csv(input_file)
    train, test = train_test_split(df, test_size=0.2, random_state=2022)

    output_train_path = f"gs://mle-dwh-torus/{output_bucket}/train.csv"
    output_test_path = f"gs://mle-dwh-torus/{output_bucket}/test.csv" 

    train.to_csv(output_train_path)
    test.to_csv(output_test_path)

    return (train, test)


def read_data(
    a: InputPath("CSV"),
): 
    print("done")

In [405]:
# import pandas as pd
# df = pd.read_csv("gs://mle-dwh-torus/raw/new_test.csv")

# df.columns = [col.lower().strip() for col in df.columns] 
# df.columns

In [411]:
import kfp

kfp.components.func_to_container_op(
    bq_extract_data,
    extra_code="from typing import NamedTuple",
    output_component_file='ingest_component.yaml', 
    base_image='gcr.io/pacific-torus-347809/mle-fp/base:latest')

kfp.components.func_to_container_op(
    basic_preprocessing,
    extra_code="from kfp.v2.dsl import Dataset, InputPath, OutputPath",
    output_component_file='basic_preprocessing_component.yaml', 
    base_image='gcr.io/pacific-torus-347809/mle-fp/base:latest',
    packages_to_install=["fsspec", "gcsfs"])

kfp.components.func_to_container_op(
    train_test_split,
    extra_code="from kfp.v2.dsl import Dataset, InputPath",
    output_component_file='tts_component.yaml', 
    base_image='gcr.io/pacific-torus-347809/mle-fp/base:latest',
    packages_to_install=["fsspec", "gcsfs", "sklearn"])

kfp.components.func_to_container_op(
    read_data,
    extra_code="from kfp.v2.dsl import Dataset, InputPath",
    output_component_file='data_component.yaml', 
    base_image='gcr.io/pacific-torus-347809/mle-fp/base:latest',
    packages_to_install=["fsspec", "gcsfs", "sklearn"])

<function Read data(a: 'CSV')>

In [412]:
ingest_op = kfp.components.load_component_from_file("./ingest_component.yaml")
remove_na_op = kfp.components.load_component_from_file("./basic_preprocessing_component.yaml")
tts_op = kfp.components.load_component_from_file("./tts_component.yaml")

data_op = kfp.components.load_component_from_file("./data_component.yaml")

PIPELINE_ROOT = "{}/pipeline/".format("gs://mle-dwh-torus")

In [413]:
# Define the pipeline
@dsl.pipeline(
   name='testing pipeline',
   description='NIL',
   pipeline_root=PIPELINE_ROOT
)
def xgboost_test_pipeline(
):
    ingest = ingest_op(
        source_project_id="pacific-torus-347809",
        source_table_url="dwh_pacific_torus.credit_card_defaults",
        destination_project_id="pacific-torus-347809",
        destination_bucket="mle-dwh-torus",
        destination_file="raw/new_test.csv",
        dataset_location="US",
        extract_job_config={},
    ) # .apply(gcp.use_gcp_secret('user-gcp-sa'))

    remove_na = remove_na_op(
        input_file=ingest.outputs["dataset_gcs_uri"],
        output_bucket="int",
        output_file="ccd2.csv"
    )
    
    tts = tts_op(
        input=remove_na.output,
        output_bucket="fin"
    )

    data = data_op(
        a=tts.outputs["train_data"]
    )




In [414]:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../pacific-torus.json"
from datetime import datetime
id = datetime.now().strftime(f"%d%H%M")

compiler.Compiler().compile(
    pipeline_func=xgboost_test_pipeline,
    pipeline_name="xgboost-train-pipeline",
    package_path="./test.json",
    type_check=True,
)

job = aip.PipelineJob(
    display_name="testpipeline",
    template_path="./test.json",
    job_id=f'test-{id}',
    pipeline_root=PIPELINE_ROOT
)

job.run()

Creating PipelineJob


/Users/c.san/opt/anaconda3/envs/mle/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1263: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


PipelineJob created. Resource name: projects/415851101207/locations/us-central1/pipelineJobs/test-232214
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/415851101207/locations/us-central1/pipelineJobs/test-232214')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/test-232214?project=415851101207
PipelineJob projects/415851101207/locations/us-central1/pipelineJobs/test-232214 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/415851101207/locations/us-central1/pipelineJobs/test-232214 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/415851101207/locations/us-central1/pipelineJobs/test-232214 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/415851101207/locations/us-central1/pipelineJobs/test-232214 current state:
PipelineState.PIPELINE_STATE_RUNNING


RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [train-test-split].; Job (project_id = pacific-torus-347809, job_id = 2427218097803886592) is failed due to the above error.; Failed to handle the job: {project_number = 415851101207, job_id = 2427218097803886592}"


In [104]:
######

In [47]:
from kfp.components import InputPath, OutputPath, create_component_from_func

def xgboost_train(
    training_data_path: InputPath('CSV'),  # Also supports LibSVM
    model_path: OutputPath('XGBoostModel'),
    model_config_path: OutputPath('XGBoostModelConfig'),
    starting_model_path: InputPath('XGBoostModel') = None,

    label_column: int = 0,
    num_iterations: int = 10,
    booster_params: dict = None,

    # Booster parameters
    objective: str = 'reg:squarederror',
    booster: str = 'gbtree',
    learning_rate: float = 0.3,
    min_split_loss: float = 0,
    max_depth: int = 6,
):
    '''Train an XGBoost model.
    Args:
        training_data_path: Path for the training data in CSV format.
        model_path: Output path for the trained model in binary XGBoost format.
        model_config_path: Output path for the internal parameter configuration of Booster as a JSON string.
        starting_model_path: Path for the existing trained model to start from.
        label_column: Column containing the label data.
        num_boost_rounds: Number of boosting iterations.
        booster_params: Parameters for the booster. See https://xgboost.readthedocs.io/en/latest/parameter.html
        objective: The learning task and the corresponding learning objective.
            See https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters
            The most common values are:
            "reg:squarederror" - Regression with squared loss (default).
            "reg:logistic" - Logistic regression.
            "binary:logistic" - Logistic regression for binary classification, output probability.
            "binary:logitraw" - Logistic regression for binary classification, output score before logistic transformation
            "rank:pairwise" - Use LambdaMART to perform pairwise ranking where the pairwise loss is minimized
            "rank:ndcg" - Use LambdaMART to perform list-wise ranking where Normalized Discounted Cumulative Gain (NDCG) is maximized
    Annotations:
        author: Alexey Volkov <alexey.volkov@ark-kun.com>
    '''
    import pandas
    import xgboost

    df = pandas.read_csv(
        training_data_path,
    )

    training_data = xgboost.DMatrix(
        data=df.drop(columns=[df.columns[label_column]]),
        label=df[df.columns[label_column]],
    )

    booster_params = booster_params or {}
    booster_params.setdefault('objective', objective)
    booster_params.setdefault('booster', booster)
    booster_params.setdefault('learning_rate', learning_rate)
    booster_params.setdefault('min_split_loss', min_split_loss)
    booster_params.setdefault('max_depth', max_depth)

    starting_model = None
    if starting_model_path:
        starting_model = xgboost.Booster(model_file=starting_model_path)

    model = xgboost.train(
        params=booster_params,
        dtrain=training_data,
        num_boost_round=num_iterations,
        xgb_model=starting_model
    )

    # Saving the model in binary format
    model.save_model(model_path)

    model_config_str = model.save_config()
    with open(model_config_path, 'w') as model_config_file:
        model_config_file.write(model_config_str)

In [48]:
create_component_from_func(
    xgboost_train,
    output_component_file='component.yaml',
    base_image='gcr.io/pacific-torus-347809/mle-fp/base:latest',
    packages_to_install=[
        'xgboost==1.6.1',
    ])

<function Xgboost train(training_data: 'CSV', starting_model: 'XGBoostModel' = None, label_column: int = '0', num_iterations: int = '10', booster_params: dict = None, objective: str = 'reg:squarederror', booster: str = 'gbtree', learning_rate: float = '0.3', min_split_loss: float = '0', max_depth: int = '6')>

In [49]:
xgboost_train_on_csv_op = kfp.components.load_component_from_file("./component.yaml")

In [78]:
@component(
    base_image="python:3.9",
    packages_to_install=["pandas", "sklearn", "fsspec", "gcsfs"],
    output_component_file="./test_process.yaml"
)
def download_from_gcs(
    gcs_url: str, 
    dataset_train: Output[Dataset],
    dataset_test: Output[Dataset],
    int_upload_url: str = None,
):
    import logging 
    import pandas as pd
    from sklearn.model_selection import train_test_split

    logger = logging.getLogger("root")
    df = pd.read_csv(gcs_url)

    columns = df.columns.to_list()
    columns = list(map(lambda col: col.lower().strip(), columns))
    df.columns = columns

    # drop NA
    df.dropna()

    train, test = train_test_split(df, test_size=0.2)

    train.to_csv(dataset_train.path)
    test.to_csv(dataset_test.path)
    logger.info("file done")
    print(dataset_test.path, dataset_train.path)


ingest_op = kfp.components.load_component_from_file("./test_process.yaml")

@dsl.pipeline(
    name="testing pipe"
)
def pipe():
    ingest_op(gcs_url="gs://mle-dwh-torus/raw/credit_cards.csv")

compiler.Compiler().compile(
    pipeline_func=pipe,
    pipeline_name="xgboost-train-pipeline",
    package_path="./test.json",
    type_check=True,
)

# job = aip.PipelineJob(
#     display_name="testpipeline",
#     template_path="./test.json",
#     job_id="test-15",
#     pipeline_root=PIPELINE_ROOT,
# )

# job.run()